# Magic Database Downloader

This simple script uses the database at [https://mtgjson.com/](!https://mtgjson.com/) and download the images from [https://scryfall.com/](!https://scryfall.com/) using the unique code provided by the database.

### Database download and import in python

In [14]:
import os

# General Variables
url_db = 'https://mtgjson.com/api/v5/AllPrintings.json'
saveLocation = './AllPrintings.json'

image_id_wildcard = '${card.identifiers.scryfallId}'
image_url = 'https://api.scryfall.com/cards/${card.identifiers.scryfallId}?format=image'

cwd = os.getcwd()
print("Current working Directory:", cwd)

Current working Directory: D:\GitRepositories\CardSorterFantasia\MagicDatabaseDownloader


In [17]:
# Download database from the site
import requests

r = requests.get(url_db, allow_redirects=True)
open(saveLocation, 'wb').write(r.content);

In [16]:
# Json reading
import json

with open(saveLocation, 'r', encoding="utf8") as json_file:
  magic_sets = json.load(json_file)
  magic_sets = magic_sets['data']

for ms in magic_sets:
  print(ms+" --> "+magic_sets[ms]['name'])

10E --> Tenth Edition
2ED --> Unlimited Edition
2XM --> Double Masters
3ED --> Revised Edition
4BB --> Fourth Edition Foreign Black Border
4ED --> Fourth Edition
5DN --> Fifth Dawn
5ED --> Fifth Edition
6ED --> Classic Sixth Edition
7ED --> Seventh Edition
8ED --> Eighth Edition
9ED --> Ninth Edition
A25 --> Masters 25
AER --> Aether Revolt
AJMP --> Jumpstart Arena Exclusives
AKH --> Amonkhet
AKR --> Amonkhet Remastered
ALA --> Shards of Alara
ALL --> Alliances
AMH1 --> Modern Horizons Art Series
ANA --> Arena New Player Experience
ANB --> Arena Beginner Set
APC --> Apocalypse
ARB --> Alara Reborn
ARC --> Archenemy
ARN --> Arabian Nights
ATH --> Anthologies
ATQ --> Antiquities
AVR --> Avacyn Restored
AZNR --> Zendikar Rising Art Series
BBD --> Battlebond
BFZ --> Battle for Zendikar
BNG --> Born of the Gods
BOK --> Betrayers of Kamigawa
BRB --> Battle Royale Box Set
BTD --> Beatdown Box Set
C13 --> Commander 2013
C14 --> Commander 2014
C15 --> Commander 2015
C16 --> Commander 2016
C17 -

In [18]:
# Properties of a card set: https://mtgjson.com/data-models/set/
print( magic_sets['ZNR'].keys() )

dict_keys(['baseSetSize', 'booster', 'cards', 'code', 'isFoilOnly', 'isOnlineOnly', 'keyruneCode', 'mtgoCode', 'name', 'releaseDate', 'tcgplayerGroupId', 'tokens', 'totalSetSize', 'translations', 'type'])


In [ ]:
print( magic_sets['ZNR']['cards'][15] )

In [8]:
from pathlib import Path

# Funzione principale incaricata di scaricare le carte
def save_edition(magic_set):

    # Vettori utilizzati per scaricare solo alcune carte
    # e tenere traccia di quelle scaricate
    color_download = {
          'B': True,
          'G': True,
          'R': True,
          'U': True,
          'W': True,
          'V': True,
      }

      rarity_download = {
          'common': color_download.copy(),
          'mythic': color_download.copy(),
          'rare': color_download.copy(), 
          'uncommon': color_download.copy(),
      }

      for i, card in enumerate(magic_set['cards']):
        rarity = card['rarity']
        color = card['colors']

        download = False

        Path("./cards_db/"+magic_set['name']+"/"+rarity).mkdir(parents=True, exist_ok=True)

        scryfallId = card['identifiers']['scryfallId']

        try:
          if len(color) == 0:
            if rarity_download[rarity]['V']: 
              rarity_download[rarity]['V'] = False
          elif len(color) > 2:
            download = True;
          elif rarity_download[rarity][color[0]]:
            download=True
            rarity_download[rarity][color[0]] = False

          if download:
            url_img = image_url.replace(image_id_wildcard, scryfallId)
            r = requests.get(url_img, allow_redirects=True)
            open("./cards_db/"+magic_set['name']+"/"+rarity+"/"+str(i)+".jpg", 'wb').write(r.content);
        except:
          print(color)

        if i%100 == 0:
          print(str(i*100/len(magic_set['cards']))+ "%")

In [ ]:
# Script totale
import os

for ms in magic_sets:
  if magic_sets[ms]['releaseDate'] > '2015-01-01':
    print(ms+" --> "+magic_sets[ms]['name'])
    save_edition(magic_sets[ms])

In [ ]:
# Save single magin set
# Function for testing
save_edition(magic_sets["MM2"])

In [ ]:
# Only google colab. This function zips all the downloaded files.
!zip -r ./cards_db.zip cards_db

In [ ]:
url_img

In [ ]:
!stat --printf="%s" cards_db.zip

In [ ]:
from google.colab import files
files.download('cards_db.zip') 